In [75]:
def Nussinov(seq, minarc=3):
    import numpy as np
    from itertools import product
    
    def is_compl(ch1, ch2):
        return 1 if {ch1, ch2} in [{'A', 'U'}, {'C', 'G'}] else 0
    
    def merge(i, j):
        max_pair=0
        max_k=0
        for k in range(i+1, j):
            curr=Pair_mat[i, k]+Pair_mat[k+1, j]
            if curr>max_pair:
                max_pair=curr
                max_k=k    
        return max_pair, max_k
    
    n=len(seq)
    
    Pair_mat=np.zeros([n, n])
    Path_mat=np.zeros([n, n])-1
    K_mat=np.zeros([n, n])-1
    
    for j in range(n):
        for i in range(j-(minarc+1),-1,-1):
            mpairs, max_k=merge(i, j)
            K_mat[i, j]=max_k
            Pair_mat[i, j]=max(Pair_mat[i+1, j], Pair_mat[i, j-1],
                              Pair_mat[i+1, j-1]+is_compl(seq[i], seq[j]),
                              mpairs)
            Path_mat[i, j]=np.argmax([Pair_mat[i+1, j], Pair_mat[i, j-1],
                              Pair_mat[i+1, j-1]+is_compl(seq[i], seq[j]),
                              mpairs])
    
    def reconstruct(i, j):
        fold=[]
        while Path_mat[i, j]!=-1:
            if Path_mat[i, j]==0:
                i+=1
            elif Path_mat[i, j]==1:
                j-=1
            elif Path_mat[i, j]==2:
                fold.append([(i, j), (seq[i], seq[j])])
                i+=1
                j-=1
            elif Path_mat[i, j]==3:
                fold.append([(K_mat[i, j], K_mat[i, j]+1),(seq[K_mat[i,j]], seq[K_mat[i,j]+1])])
                return fold+reconstruct(i, K_mat[i, j])+reconstruct(K_mat[i, j]+1, j)
            else:
                raise ValueError('Wrong Path Matrix')
        return fold    
    
    Folding=reconstruct(0, n-1)
    
    return Pair_mat, Path_mat, Folding

In [76]:
P, PM, F=Nussinov('GGACC')

In [79]:
np.max(P)

1.0

In [78]:
F

[[(0, 4), ('G', 'C')]]

In [80]:
P, PM, F=Nussinov('AAACAUGAGGAUUACCCAUGU')

In [81]:
np.max(P)

7.0

In [74]:
F

[[(2, 20), ('A', 'U')],
 [(3, 19), ('C', 'G')],
 [(4, 18), ('A', 'U')],
 [(5, 17), ('U', 'A')],
 [(6, 16), ('G', 'C')],
 [(8, 15), ('G', 'C')],
 [(9, 14), ('G', 'C')]]